In [1]:
#!pip install claptcha

In [2]:
#!rm -R data/training
#!rm -R data/validation

In [3]:
!rm -R data

rm: cannot remove 'data': No such file or directory


In [4]:
#!rmdir /Q /S "data/validation"
#!rmdir /Q /S "data/training"

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
import captcha_params
from claptcha import Claptcha
from scipy.sparse import csr_matrix
import tensorflow as tf
import os
from __future__ import print_function
import numpy as np
import cv2

np.random.seed(1337)  # for reproducibility

from keras.utils import np_utils
from keras import backend as K
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
import h5py
from keras.callbacks import ModelCheckpoint

In [6]:
N_DATASET = 200000
N_TRAINING = N_DATASET * 0.8

path_training = "data/training"
path_validation = "data/validation"

height_p = 46
width_p = 180

# input image dimensions
img_rows, img_cols = captcha_params.get_height(), captcha_params.get_width()
batch_size = 256
created_labels = {}

In [7]:
# generate  the captcha text randomly from the char lists above
def random_captcha_text(
    char_set= captcha_params.get_char_set(),
    captcha_size= captcha_params.get_captcha_size(),
):  
    good = False
    while not good:
      captcha_text = []
      lab = ""
      for i in range(captcha_size):
          c = random.choice(char_set)
          captcha_text.append(c)
          lab += c
      if created_labels.get(lab,None) is None:
        good = True
      created_labels[lab] = True
    return captcha_text

# generate the captcha text and image and save the image
def gen_captcha_text_and_image(i,path='data'):
    captcha_text = random_captcha_text()
    captcha_text = "".join(captcha_text)
    c = Claptcha(captcha_text, "font.ttf", (width_p,height_p), margin=(25,5))
    if not os.path.exists(path):  # if the folder is not existed, create it
        os.mkdir(path)
    #print(path + type_dataset + '/' + str(i) + "_" + captcha_text + ".png")
    text, _ = c.write(path + '/' + str(i) + "_" + captcha_text + ".png")
    return text

In [8]:
partition = {'train':[], 'validation':[]}
labels = {}

for i in range(N_DATASET):
    if i % 10000 == 0 and i > 0:
        print(i)
    if i < N_TRAINING:
        text = gen_captcha_text_and_image(i)
        label = text
        partition['train'].append(label)
        labels[label] = i
    else:
        text = gen_captcha_text_and_image(i)
        label = text
        partition['validation'].append(label)
        labels[label] = i

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000


In [9]:
partition_test = {'test':[]}
labels_test = {}
for i in range(10000):
  text = gen_captcha_text_and_image(i,'test')
  label = text
  partition_test['test'].append(label)
  labels_test[label] = i

In [10]:
#tf.config.list_physical_devices('GPU')

In [11]:
# the length of the captcha text
MAX_CAPTCHA = captcha_params.get_captcha_size()
# the number of elements in the char set
CHAR_SET_LEN = captcha_params.get_char_set_len()
CHAR_SET = captcha_params.get_char_set()

In [12]:
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
#
#tf.config.experimental_connect_to_cluster(tpu)
#tf.tpu.experimental.initialize_tpu_system(tpu)
#strategy = tf.distribute.experimental.TPUStrategy(tpu)
#
#print("REPLICAS: ", strategy.num_replicas_in_sync)

In [13]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=batch_size, dim=(height_p,width_p), n_channels=1,
                 n_classes= CHAR_SET_LEN * MAX_CAPTCHA, shuffle=True,test=False):
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.test = test

    def __len__(self):
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        # Initialization
        X = np.empty((self.batch_size, int(self.dim[0] / 2), 
                      int(self.dim[1] / 2), self.n_channels),
                      dtype="float32")
        #print(X.shape)
        y = np.empty((self.batch_size, MAX_CAPTCHA, CHAR_SET_LEN), dtype="uint32")
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            if self.test:
              img = cv2.imread('test/{}_{}.png'.format(self.labels[ID],ID))
            else:
              img = cv2.imread('data/{}_{}.png'.format(self.labels[ID],ID))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (int(self.dim[1] / 2), int(self.dim[0] / 2)), interpolation=cv2.INTER_AREA)
            img = np.reshape(img, (img.shape[0], img.shape[1], 1))
            #image = Image.open('data/{}_{}.png'.format(self.labels[ID],ID))
            #image = image.convert('L')
            arr = np.asarray(img, dtype="float32")
            X[i,]  = arr
            label = np.zeros((MAX_CAPTCHA, CHAR_SET_LEN))
            for j in range(MAX_CAPTCHA):
              label[j, CHAR_SET.index(ID.lower()[j])] = 1
            y[i] = label
        y_final = []
        for i in range(MAX_CAPTCHA):
          y_final.append(y[:, i, :])
        X = X.reshape(batch_size, int(img_rows/2), int(img_cols/2), 1)
        X = X.astype('float32')
        X /= 255
        return X,y_final

In [14]:
input_layer = Input((int(img_rows / 2), int(img_cols / 2), 1))
x = Conv2D(filters=32, kernel_size=(5, 5), padding='same', activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=48, kernel_size=(5, 5), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Conv2D(filters=64, kernel_size=(5, 5), padding='same', activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

x = Dropout(0.3)(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)

out = [Dense(CHAR_SET_LEN, name='digit%d' % i, activation='softmax')(x) for i in range(MAX_CAPTCHA)]
# out = Dense(num_alphabet*5, activation='sigmoid')(x)

model = Model(inputs=input_layer, outputs=out)

In [15]:
import numpy as np
from keras.models import Sequential

# Parameters
params = {'dim': (height_p,width_p),
          'batch_size': batch_size,
          'n_classes': MAX_CAPTCHA * CHAR_SET_LEN,
          'n_channels': 1,
          'shuffle': True,
          'test':False}

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

checkpoint_filepath = 'checkpoint/200k_model.ckpt'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Design model
#with strategy.scope():
#input_shape = (img_rows, img_cols, 1)
#model = load_model.get_model(input_shape)
#optimizer = keras.optimizers.Adadelta(learning_rate=0.)
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
model.summary() 

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 23, 90, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 23, 90, 32)   832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 11, 45, 32)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 11, 45, 48)   38448       max_pooling2d[0][0]              
_______________________________________________________________________________________

In [16]:
# Train model on dataset
model.fit(training_generator, validation_data=validation_generator, epochs=65, callbacks=[model_checkpoint_callback])

Epoch 1/65
625/625 [==============================] - 53s 85ms/step - loss: 0.5226 - digit0_loss: 0.0727 - digit1_loss: 0.0838 - digit2_loss: 0.0880 - digit3_loss: 0.0910 - digit4_loss: 0.0918 - digit5_loss: 0.0953 - digit0_accuracy: 0.4443 - digit1_accuracy: 0.3504 - digit2_accuracy: 0.3190 - digit3_accuracy: 0.3007 - digit4_accuracy: 0.2949 - digit5_accuracy: 0.2697 - val_loss: 0.2226 - val_digit0_loss: 0.0199 - val_digit1_loss: 0.0315 - val_digit2_loss: 0.0379 - val_digit3_loss: 0.0424 - val_digit4_loss: 0.0428 - val_digit5_loss: 0.0481 - val_digit0_accuracy: 0.8900 - val_digit1_accuracy: 0.8128 - val_digit2_accuracy: 0.7700 - val_digit3_accuracy: 0.7363 - val_digit4_accuracy: 0.7345 - val_digit5_accuracy: 0.6990
Epoch 2/65
625/625 [==============================] - 50s 80ms/step - loss: 0.2383 - digit0_loss: 0.0241 - digit1_loss: 0.0343 - digit2_loss: 0.0403 - digit3_loss: 0.0451 - digit4_loss: 0.0452 - digit5_loss: 0.0493 - digit0_accuracy: 0.8337 - digit1_accuracy: 0.7546 - digit

In [24]:
model.save("/content/drive/My Drive/Progettini/captcha_solver/model")

INFO:tensorflow:Assets written to: /content/drive/My Drive/Progettini/captcha_solver/model/assets


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# Parameters
params = {'dim': (height_p,width_p),
          'batch_size': batch_size,
          'n_classes': MAX_CAPTCHA * CHAR_SET_LEN,
          'n_channels': 1,
          'shuffle': True,
          'test':True}

# Generators
test_generator = DataGenerator(partition_test['test'], labels_test, **params)

In [19]:
score = model.evaluate(test_generator, verbose=0)

In [20]:
for i in range(len(model.metrics_names)):
  print('{}: {}'.format(model.metrics_names[i],score[i]))

loss: 0.007705009076744318
digit0_loss: 0.00038954918272793293
digit1_loss: 0.0006593515281565487
digit2_loss: 0.0006705523701384664
digit3_loss: 0.0008814713801257312
digit4_loss: 0.0011361287906765938
digit5_loss: 0.0039679561741650105
digit0_accuracy: 0.9977964758872986
digit1_accuracy: 0.9955929517745972
digit2_accuracy: 0.99609375
digit3_accuracy: 0.9948918223381042
digit4_accuracy: 0.9941906929016113
digit5_accuracy: 0.9715544581413269


In [22]:
acc = 0
tot = 0
for x,y in test_generator:
    X_test = x
    y_test = y
    predict = model.predict(X_test, batch_size=batch_size, verbose=0)
    for i in range(batch_size):
      tot += 1
      true = []
      pred = []
      for j in range(MAX_CAPTCHA):
        true.append(CHAR_SET[np.argmax(y_test[j][i])])
        pred.append(CHAR_SET[np.argmax(predict[j][i])])
      if tot % 100 == 0:
        print('TRUE: {}'.format(true))
        print('PREDICTED: {}'.format(pred))
        print('='*75)
      if true == pred:
        acc += 1
print('ACCURACY: {}'.format(acc / tot))

TRUE: ['5', 'e', 't', 'x', 'a', 'y']
PREDICTED: ['5', 'e', 't', 'x', 'a', 'y']
TRUE: ['4', '5', 'p', 'q', '3', 't']
PREDICTED: ['4', '5', 'p', 'q', '3', 't']
TRUE: ['2', 'v', 'c', 'c', '5', 'w']
PREDICTED: ['2', 'v', 'c', 'c', '5', 'w']
TRUE: ['z', '7', 'l', 'j', 'd', 'l']
PREDICTED: ['z', '7', 'l', 'j', 'd', 't']
TRUE: ['s', 'u', 'v', 'a', 'v', '2']
PREDICTED: ['s', 'u', 'v', 'a', 'v', '2']
TRUE: ['7', 'r', '0', 's', 'e', '3']
PREDICTED: ['7', 'r', '0', 's', 'e', '3']
TRUE: ['s', 'x', '0', 'k', 'x', 's']
PREDICTED: ['s', 'x', '0', 'k', 'x', 's']
TRUE: ['q', 'q', 'a', 't', '9', 'y']
PREDICTED: ['q', 'q', 'a', 't', '9', 'y']
TRUE: ['a', 'g', '8', 't', '4', 's']
PREDICTED: ['a', 'g', '8', 't', '4', 's']
TRUE: ['w', 'p', 'c', '2', 'g', 'm']
PREDICTED: ['w', 'p', 'c', '2', 'g', 'm']
TRUE: ['e', 'q', 'z', 'x', '5', 'x']
PREDICTED: ['e', 'q', 'z', 'x', '5', 'x']
TRUE: ['h', '9', 'x', '9', 'r', 'g']
PREDICTED: ['h', '9', 'x', '9', 'r', 'g']
TRUE: ['w', '3', '3', 'q', '8', 'p']
PREDICTED: ['w'